OSM export
==========
Loads a street graph and exports it in OSM format, e.g., for MATSim

In [1]:
import snman
import networkx as nx

PERIMETER = 'matsim_zrh5'

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data/'
inputs_path = data_directory + 'inputs/'
export_path = data_directory + 'outputs/' + PERIMETER + '/'
process_path = data_directory + 'process/' + PERIMETER + '/'

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load street graph')
G = snman.io.load_street_graph(process_path + 'edges_all_attributes.gpkg', process_path + 'nodes_all_attributes.gpkg')

Load street graph


In [3]:
# =====================================================================================
# PROCESS GRAPH
# =====================================================================================

print('Convert into a directed graph')
# but unlike the built-in .to_directed() method, every undirected edge corresponds to one directed edge
H = nx.MultiDiGraph()
H.nodes = G.nodes
H.graph = G.graph

for id, data in dict(G.edges).items():
    u,v,k = id
    H.add_edge(u, v, key=k, **data)

print('Organize edge directions')
# so that every one-way street is digitized in the forward direction, otherwise MATSim can not import it properly
snman.graph_tools.organize_edge_directions(H, method='by_osm_convention')

print('Update OSM tags')
# ensure that the osm tags of each edge match its lane configuration
snman.update_osm_tags(H)

Convert into a directed graph
Organize edge directions
Update OSM tags


In [4]:
# =====================================================================================
# EXPORT
# =====================================================================================

print('Export OSM file')
export_osm_tags = {
        'highway', 'maxspeed',
        'lanes', 'lanes:forward', 'lanes:backward', 'lanes:both_ways', 'oneway',
        'cycleway', 'cycleway:lane', 'cycleway:left', 'cycleway:left:lane', 'cycleway:right', 'cycleway:right:lane',
        'bus:lanes:backward', 'bus:lanes:forward', 'vehicle:lanes:backward', 'vehicle:lanes:forward',
        '_connected_component'
    }

snman.io.export_osm_xml(H, export_path + 'osm.osm', export_osm_tags, uv_tags=True, tag_all_nodes=False)

Export OSM file
